In [0]:
bronze_path = "/mnt/bronze"
silver_path = "/mnt/silver"

print(f"Caminho da camada Bronze: {bronze_path}")
print(f"Caminho da camada Silver: {silver_path}")

In [0]:
display(dbutils.fs.ls("/mnt/bronze"))


In [0]:
from pyspark.sql.functions import col, to_timestamp

# Ler os dados Parquet da camada bronze
clientes_df = spark.read.parquet(f"{bronze_path}/clientes.parquet")

# Transformações
clientes_silver_df = clientes_df.withColumn("data_cadastro_ts", to_timestamp(col("data_cadastro"))) \
                                .drop("data_cadastro") \
                                .withColumnRenamed("data_cadastro_ts", "data_cadastro") \
                                .dropDuplicates(["cliente_id"])

# Escrever na silver
clientes_silver_df.write.mode("overwrite").format("parquet").save(f"{silver_path}/clientes")

print("Tabela de clientes salva na camada Silver.")
display(clientes_silver_df)

In [0]:
# Ler os dados de pedidos
pedidos_df = spark.read.parquet(f"{bronze_path}/pedidos.parquet")

# Transformações: converter datas, garantir consistência
pedidos_silver_df = pedidos_df.withColumn("data_pedido_ts", to_timestamp(col("data_pedido"))) \
                              .drop("data_pedido") \
                              .withColumnRenamed("data_pedido_ts", "data_pedido") \
                              .dropDuplicates(["pedido_id"])

# Escrever silver
pedidos_silver_df.write.mode("overwrite").format("parquet").save(f"{silver_path}/pedidos")

print("Tabela de pedidos salva na camada Silver.")
display(pedidos_silver_df)

In [0]:
# Ler os dados de pagamentos
pagamentos_df = spark.read.parquet(f"{bronze_path}/pagamentos.parquet")


pagamentos_silver_df = pagamentos_df.withColumn("data_pagamento_ts", to_timestamp(col("data_pagamento"))) \
                                    .drop("data_pagamento") \
                                    .withColumnRenamed("data_pagamento_ts", "data_pagamento") \
                                    .dropDuplicates(['pagamento_id'])

# Escrever na silver
pagamentos_silver_df.write.mode("overwrite").format("parquet").save(f"{silver_path}/pagamentos")

print("Tabela de pagamentos salva na camada Silver.")
display(pagamentos_silver_df)

In [0]:
from pyspark.sql.functions import when

# Ler os dados de entregas
entregas_df = spark.read.parquet(f"{bronze_path}/entregas.parquet")

# Transformações e Enriquecimento
entregas_silver_df = entregas_df.withColumn("data_envio_ts", to_timestamp(col("data_envio"))) \
                                .withColumn("data_estimada_entrega_ts", to_timestamp(col("data_estimada_entrega"))) \
                                .withColumn("data_entrega_real_ts", to_timestamp(col("data_entrega_real"))) \
                                .drop("data_envio", "data_estimada_entrega", "data_entrega_real") \
                                .withColumnRenamed("data_envio_ts", "data_envio") \
                                .withColumnRenamed("data_estimada_entrega_ts", "data_estimada_entrega") \
                                .withColumnRenamed("data_entrega_real_ts", "data_entrega_real") \
                                .dropDuplicates(['entrega_id'])

# ENRIQUECIMENTO: Adicionar uma coluna para identificar entregas atrasadas (SLA)
entregas_silver_df = entregas_silver_df.withColumn(
    "sla_atrasado",
    when(col("data_entrega_real") > col("data_estimada_entrega"), True).otherwise(False)
)

# Escrever na camada silver
entregas_silver_df.write.mode("overwrite").format("parquet").save(f"{silver_path}/entregas")

print("Tabela de entregas salva e enriquecida na camada Silver.")
display(entregas_silver_df)